# **Interpolation in image processing**

<div style="color:#777777;margin-top: -15px;">
<b>Author</b>: Norman Juchler |
<b>Course</b>: ADLS ISP |
<b>Version</b>: v1.2 <br><br>
<!-- Date: 03.04.2025 -->
<!-- Comments: Fully refactored. -->
</div>

Interpolation is a method used to estimate the value of a function at a given point based on known values at surrounding points. In image processing, we can use interpolation to estimate pixel values at non-integer coordinates using the known pixel values at integer coordinates. This is commonly needed in tasks such as resizing, rotating, and warping images. In OpenCV, interpolation is used in functions like: `cv2.resize()`, `cv2.warpAffine()`, and `cv2.warpPerspective()`.

To apply interpolation to 2D image data, the concept must be extended from 1D to 2D (or higher dimensions). Figure 1 shows a comparison between 1D and 2D interpolation. Figure 2 illustrates how different interpolation methods affect a 2D grid.

<center>
<img src="../data/doc/interpolation-comparison-2d-3d-plus.svg" alt="interpolation-comparison-2d-3d-plus" width="600"/>

**Figure 1** Comparison of different interpolation techniques in 1D and 2D. [Source](https://en.wikipedia.org/wiki/Bicubic_interpolation)
</center>
<br><br>

![interpolation-methods-2d](../data/doc/interpolation-methods-2d.svg)

<br>

Common Interpolation Methods in OpenCV

The interpolation method in OpenCV functions is specified using a flag from [`cv.InterpolationFlags`](https://docs.opencv.org/4.x/da/d54/group__imgproc__transform.html#ga5bb5a1fea74ea38e1a5445ca803ff121). Common options include:

* `INTER_NEAREST`: Nearest-neighbor interpolation. Very fast, but can produce blocky or jagged results.
* `INTER_LINEAR`: [Bilinear interpolation](https://en.wikipedia.org/wiki/Bilinear_interpolation). Still fast, with smoother results than nearest-neighbor.
* `INTER_CUBIC`: [Bicubic interpolation](https://en.wikipedia.org/wiki/Bicubic_interpolation). Slower, but produces smoother and more natural-looking results, especially when stretching images.
* `INTER_AREA`: Uses pixel area relation. Best suited for image shrinking. Uses (windowed/boxed) averaging to reduce aliasing.
* `INTER_LANCZOS`: Lanczos interpolation (using an 8×8 pixel neighborhood). Slower, but good for preserving detail in high-resolution images.

Lanczos interpolation works by fitting a sinc function (specifically a windowed sinc) over a neighborhood of surrounding pixels—typically an 8×8 grid. It provides high-quality results, especially for resizing images with fine details, by minimizing aliasing and preserving sharp edges.

Note: In high-resolution images, differences between interpolation methods are subtle and often only visible when zoomed in.

---

## **Preparations**

The usual preparations... The package `isp` provides some helper functions to easily render images in this Jupyter notebook.

In [ ]:
import sys
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

# Enable vectorized output (for nicer plots)
%config InlineBackend.figure_formats = ["svg"]

# Inline backend configuration
%matplotlib inline

# Functionality related to this course
sys.path.append("..")
import isp

# Jupyter / IPython configuration:
# Automatically reload modules when modified
%load_ext autoreload
%autoreload 2

In addition to loading images from files, we will also use synthetically generated checkerboard patterns – a common test pattern in computer vision tasks. (You may recall implementing your own version of this in a previous exercise.)



In [ ]:
def create_checkerboard_image(width, height, tile_size, rotation=0):
    """
    Create a synthetic checkerboard image.

    Parameters:
        width (int): Width of the image in pixels.
        height (int): Height of the image in pixels.
        tile_size (int): Size of each checker tile in pixels.
        rotation (float): Optional rotation angle in degrees (default: 0).

    Returns:
        np.ndarray: Grayscale checkerboard image of shape (height, width).
    """
    
    # Number of tiles needed
    tiles_x = width // tile_size + 1
    tiles_y = height // tile_size + 1

    # Create 2x2 checker tile
    tile_white = np.ones((tile_size, tile_size), dtype=np.uint8)
    tile_black = np.zeros((tile_size, tile_size), dtype=np.uint8)
    
    # Create a 2x2 checkerboard tile
    checker_tile = np.vstack([np.hstack([tile_white, tile_black]),
                              np.hstack([tile_black, tile_white])]) 
    checker_tile *= 255
    
    # Create the full checkerboard by tiling the 2x2 tile
    checkerboard = np.tile(checker_tile, (tiles_y // 2, tiles_x // 2))

    # Crop to exact dimensions
    checkerboard = checkerboard[:height, :width]

    # Rotate if needed
    if rotation != 0:
        center = (width // 2, height // 2)
        rot_mat = cv.getRotationMatrix2D(center, rotation, 1.0)
        checkerboard = cv.warpAffine(checkerboard, rot_mat, (width, height),
                                     flags=cv.INTER_LINEAR,
                                     borderMode=cv.BORDER_WRAP)

    # Optionally convert to 3-channel for visualization
    # checkerboard = cv.cvtColor(checkerboard, cv.COLOR_GRAY2BGR)

    return checkerboard

Next, we define a function to compare different interpolation methods using `cv.resize()`. While we focus on this function for simplicity, the same interpolation options are available in other OpenCV functions. It's helpful to understand how `cv.resize()` works before proceeding to the next cell.

In [ ]:
interpolation_methods = {
    "nearest": cv.INTER_NEAREST,
    "bilinear": cv.INTER_LINEAR,
    "bicubic": cv.INTER_CUBIC,
    "area": cv.INTER_AREA,
    "lanczos4": cv.INTER_LANCZOS4
}


def interpolation_test(img, scale, ncols=-1, figsize=(9, 1.8)):
    """
    Resize an image using various interpolation methods and display the results.
    """
    height, width = img.shape[:2]
    new_height, new_width = int(height * scale), int(width * scale)

    # Determine color map for grayscale images
    is_gray = img.ndim == 2 or (img.ndim == 3 and img.shape[2] == 1)
    cmap = "gray" if is_gray else None

    print(f"Original size: {height} x {width}")
    print(f"Resized size:  {new_height} x {new_width}")

    results = []
    for method in interpolation_methods.values():
        resized = cv.resize(img, (new_width, new_height), interpolation=method)
        results.append(resized)

    # Include original image as first
    results.insert(0, img)
    
    # Labels for the images
    titles = ["original"] + list(interpolation_methods.keys())

    if ncols < 0:
        ncols = len(titles)

    isp.show_image_grid(
        results,
        titles=titles,
        ncols=ncols,
        suppress_info=True,
        figsize=figsize,
        shape=None
    )

    return results

## **Downscaling**

For downscaling, area-based interpolation is a good choice. Lanczos interpolation can help reduce aliasing artifacts, but its effectiveness varies depending on the image content and scaling factor.

In [ ]:

scale = 0.21

isp.show_header("Test 1: Downscale cat image")
image = cv.imread(filename="../data/images/cat.png", flags=cv.IMREAD_COLOR)
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
out = interpolation_test(image, scale=scale)

isp.show_header("Test 2: Downscale checkerboard")
image = create_checkerboard_image(200, 200, 25, rotation=20)
out = interpolation_test(image, scale=scale)

## **Upscaling**

For upscaling, bicubic and Lanczos interpolation generally produce the best results. Nearest-neighbor and area-based methods tend to introduce blocky artifacts, while bilinear interpolation often appears slightly blurry.

In [ ]:
isp.show_header("Test 3: Upscale cat image")
image = cv.imread(filename="../data/images/cat_low.png",
                   flags=cv.IMREAD_COLOR)
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
out = interpolation_test(image, scale=4)

isp.show_header("Test 4: Upscale checkerboard")
image = create_checkerboard_image(60, 60, 10, rotation=30)
out = interpolation_test(image, scale=4)